# 🧠 Model Training

This notebook is designed to be deployed into a containerized node.

## 1. Python Setup

In [ ]:
%python -m venv ultralytics-env

In [ ]:
%. /ultralytics-env/Scripts/activate

In [ ]:
%pip install gdown

In [ ]:
%pip install dotenv

In [ ]:
%pip install ultralytics

In [ ]:
%pip install pandas

In [ ]:
%pip install numpy

In [ ]:
from ultralytics import YOLO
import gdown
from dotenv import load_dotenv
import os